<a href="https://colab.research.google.com/github/Skanda-Mahesh/AI-lab/blob/main/Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
def unify(psi1, psi2, subst=None):
    if subst is None:
        subst = {}

    def apply_subst(s_map, expr):
        if not s_map:
            return expr
        if isinstance(expr, str) and expr.startswith('?'):
            return s_map.get(expr, expr)
        elif isinstance(expr, tuple):
            return tuple([apply_subst(s_map, arg) for arg in expr])
        return expr

    def is_variable(expr):
        return isinstance(expr, str) and expr.startswith('?')

    # Apply current substitution before unification
    _psi1 = apply_subst(subst, psi1)
    _psi2 = apply_subst(subst, psi2)

    # Step 1: Base Case (Variable or Constant)
    if is_variable(_psi1) or is_variable(_psi2) or not isinstance(_psi1, tuple) or not isinstance(_psi2, tuple):
        if _psi1 == _psi2:
            return subst
        elif is_variable(_psi1):
            if _psi1 in str(_psi2):
                return 'FAILURE'
            return {**subst, _psi1: _psi2}
        elif is_variable(_psi2):
            if _psi2 in str(_psi1):
                return 'FAILURE'
            return {**subst, _psi2: _psi1}
        else:
            return 'FAILURE'

    # Step 2 & 3: Check Predicate and Arity
    if _psi1[0] != _psi2[0] or len(_psi1) != len(_psi2):
        return 'FAILURE'

    # Step 5: Process Arguments
    for i in range(1, len(_psi1)):
        arg1 = _psi1[i]
        arg2 = _psi2[i]

        s = unify(arg1, arg2, subst)

        if s == 'FAILURE':
            return 'FAILURE'

        # Update the substitution for the next iteration
        subst = s

    # Step 6: Return SUBST
    return subst

examples = [
    # 1. p(b,X,f(g(Z))) and p(z,f(Y),f(Y)) -> {?z: b, ?X: f(g(?Z)), ?Y: g(?Z)} (Variables normalized to start with '?')
    (('p', 'b', '?X', ('f', ('g', '?Z'))), ('p', '?z', ('f', '?Y'), ('f', '?Y'))),

    # 2. Q(a,g(x,a),f(y)) and Q(a,g(f(b),a),x) -> {?x: f(b), ?y: b}
    (('Q', 'a', ('g', '?x', 'a'), ('f', '?y')), ('Q', 'a', ('g', ('f', 'b'), 'a'), '?x')),

    # 3. p(f(a),g(Y)), p(X,X) -> FAILURE
    (('p', ('f', 'a'), ('g', '?Y')), ('p', '?X', '?X')),

    # 4. prime(11) and prime(y) -> {?y: 11}
    (('prime', 11), ('prime', '?y')),

    # 5. knows(John,x),knows(y,mother(y)) -> {?y: John, ?x: mother(John)}
    (('knows', 'John', '?x'), ('knows', '?y', ('mother', '?y'))),

    # 6. knows(John,x),knows(y,Bill) -> {?y: John, ?x: Bill}
    (('knows', 'John', '?x'), ('knows', '?y', 'Bill')),
]

print("--- Unification Algorithm Examples ---")

for i, (exp1, exp2) in enumerate(examples):
    # Normalize integer arguments to strings for consistent output,
    # but the implementation handles them fine as constants.
    exp1_str = str(exp1).replace("'", "").replace("(", "").replace(")", "").replace(", ", ",")
    exp2_str = str(exp2).replace("'", "").replace("(", "").replace(")", "").replace(", ", ",")

    result = unify(exp1, exp2)
    print(f"\nExample {i+1}:")
    print(f"  Unify: {exp1_str} and {exp2_str}")

    # Format the result for cleaner printing
    if result == 'FAILURE':
        print(f"  Result: FAILURE")
    else:
        # Convert the dictionary output to the standard substitution notation {term/variable}
        formatted_subst = []
        for var, term in result.items():
           # Only show substitutions for variables that are not already bound to themselves
            if var != term:
                 # Standardize term representation for display
                term_str = str(term).replace("'", "").replace("(", "").replace(")", "").replace(", ", ",")
                formatted_subst.append(f"{term_str}/{var}")

        # The result from the code is the accumulated dictionary.
        print(f"  MGU: {{{', '.join(formatted_subst)}}}")

--- Unification Algorithm Examples ---

Example 1:
  Unify: p,b,?X,f,g,?Z and p,?z,f,?Y,f,?Y
  MGU: {b/?z, f,?Y/?X, g,?Z/?Y}

Example 2:
  Unify: Q,a,g,?x,a,f,?y and Q,a,g,f,b,a,?x
  MGU: {f,b/?x, b/?y}

Example 3:
  Unify: p,f,a,g,?Y and p,?X,?X
  Result: FAILURE

Example 4:
  Unify: prime,11 and prime,?y
  MGU: {11/?y}

Example 5:
  Unify: knows,John,?x and knows,?y,mother,?y
  MGU: {John/?y, mother,John/?x}

Example 6:
  Unify: knows,John,?x and knows,?y,Bill
  MGU: {John/?y, Bill/?x}
